In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load and preprocess the data

In [3]:
data_dir = "../data/images_train/"

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, # these parameters can be tuned
    width_shift_range=0.2, # these parameters can be tuned
    height_shift_range=0.2, # these parameters can be tuned
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32, # batch size can be tuned
    class_mode='binary',
    subset='training'  # Set as training data
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

Found 493 images belonging to 2 classes.


Found 123 images belonging to 2 classes.


# Load a pre-trained ResNet50 model

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x) # dropout can be tuned
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), # learning rate can be tuned
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10, # epochs can be tuned
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/10


15/15 [==============================] - 58s 4s/step - loss: 0.7505 - accuracy: 0.5380 - val_loss: 0.6604 - val_accuracy: 0.5417
Epoch 2/10
15/15 [==============================] - 54s 4s/step - loss: 0.7036 - accuracy: 0.5727 - val_loss: 0.6668 - val_accuracy: 0.5417
Epoch 3/10
15/15 [==============================] - 53s 4s/step - loss: 0.7033 - accuracy: 0.5770 - val_loss: 0.6266 - val_accuracy: 0.6979
Epoch 4/10
15/15 [==============================] - 54s 4s/step - loss: 0.6588 - accuracy: 0.6291 - val_loss: 0.6262 - val_accuracy: 0.7292
Epoch 5/10
15/15 [==============================] - 54s 4s/step - loss: 0.6132 - accuracy: 0.6725 - val_loss: 0.6176 - val_accuracy: 0.6771
Epoch 6/10
15/15 [==============================] - 55s 4s/step - loss: 0.6336 - accuracy: 0.6508 - val_loss: 0.6327 - val_accuracy: 0.6667
Epoch 7/10
15/15 [==============================] - 60s 4s/step - loss: 0.6156 - accuracy: 0.6729 - val_loss: 0.5616 - val_accuracy: 0.7083
Epoch 8/10
15/15 [

In [9]:
test_dir = '../data/images_test'

predictions = []

for image_name in os.listdir(test_dir):
    image_id = image_name[:3]
    img_path = os.path.join(test_dir, image_name)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.

    # predict
    prediction = model.predict(img_array)
    predicted_class = int(prediction[0][0] <= 0.5) # 0 for kitchen, 1 for bathroom
    predictions.append([image_id,predicted_class])

predictions_df = pd.DataFrame(predictions, columns=['id', 'class'])
predictions_df.to_csv('image_prediction.csv', index=False)

1/1 [==============================] - 0s 68ms/step


In [ ]:
# train_generator.class_indices

{'bathroom': 0, 'kitchen': 1}
